In [145]:
import pandas as pd

In [147]:
import os
import re 
cnt = 0
data = []
for fname in os.listdir():
    
    if fname.endswith('.csv') and fname.startswith('('):
        
        with open(fname) as f:
            
            keys = f.readline().strip().split(',')
            for line in f:
                temp = {}
                if len(line.split(',')) < 7:
                    print(fname)
                    continue
                temp['Origin'] = line.split('.csv')[0]
                temp['Subject'] = line.split(',')[2]
                temp['Object'] = line.split(',')[3]
                temp['Relation'] = line.split(',')[4]
                
               

                temp['NLIlabel'] = line.split(',')[-3]
                temp['NLIscore'] = line.split(',')[-2]

                if temp['NLIlabel'] not in ('NEUTRAL','CONTRADICTION','ENTAILMENT'):
                    continue
                #print(temp['Relation'])
                #print(temp['NLIlabel'])
                try:
                    result = re.search(f"{temp['Relation']}(.*?){temp['NLIlabel']}",line)
                except Exception as e:
                    print(e)
                    print(line)
                    continue
                if result and result.group(1):
                    temp['OrigSent'] = result.group(1).lstrip(',').rstrip(',')
                else:
                    temp['OrigSent'] = ''
                #print(temp)
                
                
                data.append(temp)
                cnt += 1

nothing to repeat at position 0
('Book 3: Prisoner of Azkaban', 'chap-22'),*ignored her,member of,*ignored her,Ron *ignored her.  ,NEUTRAL,0.8526767492294312,

nothing to repeat at position 0
('Book 2: Chamber of Secrets', 'chap-19'),I'm so glad I've got this diary to *309*,has part,*309*,I'm so glad I've got this diary to  *309*  confide in ....,ENTAILMENT,0.512889564037323,



In [148]:
df = pd.DataFrame(data)

In [149]:
df['NLIlabel'].value_counts()

NLIlabel
NEUTRAL          86697
ENTAILMENT       53699
CONTRADICTION    12120
Name: count, dtype: int64

In [150]:
df_filtered = df[(df['NLIlabel'] == 'ENTAILMENT' ) & (df['NLIscore'] > '0.7') & (df['Subject'] != df['Object']) & (df['Object'] != df['Relation']) & (df['Subject'] != df['Relation'])]

In [127]:
import requests
res = requests.get("https://raw.githubusercontent.com/gastonstat/harry-potter-data/main/csv-data-file/harry_potter_books.csv")
from io import StringIO

import pandas as pd

TESTDATA = StringIO(res.text)

df_ner = pd.read_csv(TESTDATA, sep=",")
df_ner.head()

agg_df = df_ner.groupby(['book','chapter']).text.apply(lambda x:" ".join(x))

In [128]:
corpus = " ".join(agg_df.values)

In [129]:
# Load SpaCy model for NER
from spacy.cli import download
import spacy
download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# Text of length 6269985 exceeds maximum of 1000000.
batch = len(corpus) // 900000

entities_set = set()
for i in range(0, batch):
    batch_corpus = corpus[i*900000:(i+1)*900000]
    doc = nlp(batch_corpus)
    # Extract named entities
    entities_list = [(entity.text, entity.label_) for entity in doc.ents]
    filtered_entity = ['DATE','WORK_OF_ART','CARDINAL','TIME','ORDINAL']
    entities = set([s[0] for s in entities_list if s[1] not in filtered_entity ])
    for entity in entities:
        entities_set.add(entity)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.8 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [151]:
isin = df_filtered.isin(entities_set)

In [152]:
df_filtered[isin[['Subject','Object']].sum(axis=1) > 0].to_csv('KG.csv')